In [1]:
#importing Lib


In [2]:
import pandas as pd
import numpy as np

In [3]:
# Load dataset

In [4]:
RegTech = pd.read_csv('PS_2017_log.csv', low_memory=False)


In [5]:
RegTech.head(5)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [6]:
RegTech.tail(5)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
1048570,95,CASH_OUT,132557.35,C1179511630,479803.00,347245.65,C435674507,484329.37,616886.72,0,0
1048571,95,PAYMENT,9917.36,C1956161225,90545.00,80627.64,M668364942,0.00,0.00,0,0
1048572,95,PAYMENT,14140.05,C2037964975,20545.00,6404.95,M1355182933,0.00,0.00,0,0
1048573,95,PAYMENT,10020.05,C1633237354,90605.00,80584.95,M1964992463,0.00,0.00,0,0
1048574,95,PAYMENT,11450.03,C1264356443,80584.95,69134.92,M677577406,0.00,0.00,0,0


In [7]:
# Shape of dataset

In [8]:
print("Dataset shape:", RegTech.shape)

Dataset shape: (1048575, 11)


In [9]:
# Cheking datatypes and null values 

In [10]:
RegTech.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 11 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   step            1048575 non-null  int64  
 1   type            1048575 non-null  object 
 2   amount          1048575 non-null  float64
 3   nameOrig        1048575 non-null  object 
 4   oldbalanceOrg   1048575 non-null  float64
 5   newbalanceOrig  1048575 non-null  float64
 6   nameDest        1048575 non-null  object 
 7   oldbalanceDest  1048575 non-null  float64
 8   newbalanceDest  1048575 non-null  float64
 9   isFraud         1048575 non-null  int64  
 10  isFlaggedFraud  1048575 non-null  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 88.0+ MB


In [11]:
# Statistical summary

In [12]:
RegTech.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1048575.0
mean,2.696617e+01,1.586670e+05,8.740095e+05,8.938089e+05,9.781600e+05,1.114198e+06,1.089097e-03,0.0
std,1.562325e+01,2.649409e+05,2.971751e+06,3.008271e+06,2.296780e+06,2.416593e+06,3.298351e-02,0.0
min,1.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
25%,1.500000e+01,1.214907e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
50%,2.000000e+01,7.634333e+04,1.600200e+04,0.000000e+00,1.263772e+05,2.182604e+05,0.000000e+00,0.0
75%,3.900000e+01,2.137619e+05,1.366420e+05,1.746000e+05,9.159235e+05,1.149808e+06,0.000000e+00,0.0
max,9.500000e+01,1.000000e+07,3.890000e+07,3.890000e+07,4.210000e+07,4.220000e+07,1.000000e+00,0.0


In [13]:
RegTech.columns

Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud',
       'isFlaggedFraud'],
      dtype='object')

In [14]:
#Basic EDA

In [15]:
# Unique transaction types
print(RegTech['type'].value_counts())

# Distribution of amounts
RegTech['amount'].describe()

# Check for missing values
RegTech.isnull().sum()

# Fraud vs non-fraud count
RegTech['isFraud'].value_counts(normalize=True)


type
CASH_OUT    373641
PAYMENT     353873
CASH_IN     227130
TRANSFER     86753
DEBIT         7178
Name: count, dtype: int64


isFraud
0    0.998911
1    0.001089
Name: proportion, dtype: float64

In [16]:
#creatinh initial compliance rules based on common regulatory patterns

### Step 1: Adding  Rule-Based Flags

#Copying dataframe to avoide overwritting

In [17]:
RegTech_rules = RegTech.copy()

#### Rule N0 1:- High amount Transactions (e.g>$10000)

In [18]:
RegTech_rules['rule_high_amount'] = RegTech_rules['amount'] > 10000

#### Rule No 2: Multiple transfer by the same sender in a short time 

In [19]:
RegTech_rules['rule_frequent_sender'] = RegTech_rules.duplicated(subset=['nameOrig', 'step'], keep=False)

#### Rule 3: Cash-out immediately after transfer (possible layering)

In [20]:
RegTech_rules['rule_cashout_after_transfer'] = (
    (RegTech_rules['type'] == 'CASH_OUT') &
    (RegTech_rules['nameOrig'].isin(RegTech_rules.loc[RegTech_rules['type'] == 'TRANSFER', 'nameDest']))
)

#### Combine Rule flag 

In [21]:
RegTech_rules['rule_flagged'] = RegTech_rules[['rule_high_amount', 'rule_frequent_sender', 'rule_cashout_after_transfer']].any(axis=1)

#### Check how many were flagged by rules


In [22]:
RegTech_rules['rule_flagged'].value_counts(normalize=True)

rule_flagged
True     0.783973
False    0.216027
Name: proportion, dtype: float64

### Step 2 Inspecting rule effectiveness

In [23]:
# Check how many records each rule flagged
rule_counts = RegTech_rules[['rule_high_amount', 'rule_frequent_sender', 'rule_cashout_after_transfer']].sum()
# Percentage contribution
rule_percentage = (rule_counts / len(RegTech_rules)) * 100
print(rule_percentage)

rule_high_amount               78.397158
rule_frequent_sender            0.002289
rule_cashout_after_transfer     0.000381
dtype: float64


### Step 3: Compare Rule Flags vs Actual Non-Compliance (isFraud)

In [24]:
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(RegTech_rules['isFraud'], RegTech_rules['rule_flagged']))
print(classification_report(RegTech_rules['isFraud'], RegTech_rules['rule_flagged']))


[[226464 820969]
 [    56   1086]]
              precision    recall  f1-score   support

           0       1.00      0.22      0.36   1047433
           1       0.00      0.95      0.00      1142

    accuracy                           0.22   1048575
   macro avg       0.50      0.58      0.18   1048575
weighted avg       1.00      0.22      0.36   1048575



##### Interpretation:

######  Recall (1 → 0.95): Your rules catch almost all true non-compliance cases — great for compliance (high recall).

###### Precision (1 → 0.00): But they flag too many false positives (most flagged transactions aren’t actually non-compliant).

###### Accuracy (0.22): Not meaningful here — compliance tasks value recall over accuracy.

###### Conclusion: This is exactly how rule-based systems behave in real-world RegTech-very high recall but poor precision.

### Feature Engineering 

##### create new engineered features to feed into an ML model that learns subtle compliance patterns

In [25]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

RegTech_feature = RegTech_rules.copy()

# Encode transaction type
le = LabelEncoder()
RegTech_feature['type_encoded'] = le.fit_transform(RegTech_feature['type'])

# Compute ratio features
RegTech_feature['amount_ratio'] = np.where(RegTech_feature['oldbalanceOrg'] != 0,
                                 RegTech_feature['amount'] / RegTech_feature['oldbalanceOrg'], 0)

RegTech_feature['balance_diff'] = RegTech_feature['oldbalanceOrg'] - RegTech_feature['newbalanceOrig']

# Frequency of transactions per sender
txn_freq = RegTech_feature.groupby('nameOrig')['step'].transform('count')
RegTech_feature['txn_frequency'] = txn_freq

# Average amount per sender
avg_amt = RegTech_feature.groupby('nameOrig')['amount'].transform('mean')
RegTech_feature['avg_amount_sender'] = avg_amt

# Select features and target
features = ['amount', 'type_encoded', 'amount_ratio', 'balance_diff',
            'txn_frequency', 'avg_amount_sender', 'rule_high_amount']
target = 'isFraud'

X = RegTech_feature[features]
y = RegTech_feature[target]


In [26]:
#### Model Training 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Train baseline Random Forest
rf = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
rf.fit(X_train, y_train)

# Predictions
y_pred = rf.predict(X_test)

# Evaluate
print(classification_report(y_test, y_pred))


In [ ]:
### Model is overfitting so  solving this by explroring data leakage 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Check correlation with target
corr = RegTech_feature[features + [target]].corr()
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.show()


In [ ]:
# Evaluating  on a Smaller Hold-Out Set

In [ ]:
from sklearn.metrics import roc_auc_score, confusion_matrix

# Predicted probabilities
y_prob = rf.predict_proba(X_test)[:,1]

print("ROC-AUC:", roc_auc_score(y_test, y_prob))
print(confusion_matrix(y_test, y_pred))


In [ ]:
#### Explanability 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

feat_importance = pd.Series(rf.feature_importances_, index=features).sort_values(ascending=False)
feat_importance.plot(kind='bar', title='Feature Importance')
plt.show()


## Creating Dashboards 

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


In [ ]:
###### Generate Prediction Probabilities

In [ ]:
# Predict probabilities for risk scoring
RegTech_feature['risk_score'] = rf.predict_proba(X)[:, 1]

# Sort by risk score descending
RegTech_top_risk = RegTech_feature.sort_values('risk_score', ascending=False)
RegTech_top_risk.head(10)


In [ ]:
##### – Visualize Risk Distribution

In [ ]:
fig = px.histogram(RegTech_feature, x='risk_score', nbins=50, title='Distribution of Risk Scores',
                   labels={'risk_score': 'Predicted Compliance Risk'},
                   color_discrete_sequence=['#636EFA'])
fig.show()


In [ ]:
##### Top Risky Transactions Table

In [ ]:
RegTech_top10 = RegTech_top_risk[['nameOrig', 'type', 'amount', 'oldbalanceOrg',
                        'newbalanceOrig', 'risk_score']].head(10)

fig = go.Figure(data=[go.Table(
    header=dict(values=list(RegTech_top10.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[RegTech_top10[col] for col in RegTech_top10.columns],
               fill_color='lavender',
               align='left'))
])

fig.update_layout(title='Top 10 High-Risk Transactions')
fig.show()


In [ ]:
##### Feature importance 

In [ ]:
feat_importance = pd.Series(rf.feature_importances_, index=features).sort_values(ascending=True)
fig = px.bar(
    x=feat_importance.values, 
    y=feat_importance.index,
    orientation='h',
    title='Feature Importance in Compliance Model',
    labels={'x': 'Importance', 'y': 'Feature'},
    color=feat_importance.values,
    color_continuous_scale='Blues'
)
fig.show()


In [ ]:
##### Risk by transaction type 

In [ ]:
fig = px.box(RegTech_feature, x='type', y='risk_score',
             title='Risk Score by Transaction Type',
             color='type')
fig.show()


In [ ]:
#pip install jupyter-dash plotly dash


In [ ]:
print(type(rf))
print(type(RegTech_feature))
print(type(features))


In [ ]:
rf.predict_proba(X)[:, 1]


In [ ]:
X = RegTech_feature[features]


In [ ]:
rf


In [ ]:
df_dashboard['risk_score'] = rf.predict_proba(X)[:, 1]


In [ ]:
if rf is not None and 'predict_proba' in dir(rf):
    X = RegTech_feature[features]
    df_dashboard['risk_score'] = rf.predict_proba(X)[:, 1]
else:
    raise ValueError("Model `rf` is not fitted or not available.")


In [ ]:
# Install these once (uncomment if not already installed)
# !pip install jupyter-dash dash plotly

from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd

# --------------------------------------------------------------------
# Prepare Data
# --------------------------------------------------------------------
df_dashboard = RegTech_feature.copy()

# Create risk levels based on model scores
df_dashboard['risk_score'] = rf.predict_proba(X)[:, 1]
df_dashboard['risk_level'] = pd.cut(
    df_dashboard['risk_score'],
    bins=[0, 0.3, 0.7, 1],
    labels=['Low', 'Medium', 'High']
)

# --------------------------------------------------------------------
# App Initialization
# --------------------------------------------------------------------
app = JupyterDash(__name__)

# Colors & Theme
colors = {
    'background': '#0E1117',
    'card': '#1E2130',
    'text': '#E8E9EB',
    'accent1': '#636EFA',
    'accent2': '#EF553B',
    'accent3': '#00CC96'
}

# --------------------------------------------------------------------
# Dashboard Layout
# --------------------------------------------------------------------
app.layout = html.Div(style={'backgroundColor': colors['background'], 'padding': '20px'}, children=[

    html.H1("💼 RegTech Compliance Monitoring Dashboard",
            style={'textAlign': 'center', 'color': colors['accent1']}),

    html.Hr(style={'borderColor': colors['accent1']}),

    # Dropdown Filter
    html.Div([
        html.Label("Select Transaction Type:", style={'color': colors['text'], 'fontSize': 16}),
        dcc.Dropdown(
            id='type_filter',
            options=[{'label': t, 'value': t} for t in df_dashboard['type'].unique()],
            value='CASH_OUT',
            multi=False,
            style={'color': '#000'}
        )
    ], style={'width': '40%', 'margin': 'auto'}),

    html.Br(),

    # KPI Summary
    html.Div(id='summary_cards', style={'display': 'flex', 'justifyContent': 'space-around',
                                        'color': colors['text'], 'marginBottom': '25px'}),

    # Dashboard Charts
    html.Div([
        dcc.Graph(id='risk_dist'),
        dcc.Graph(id='top_risk_table'),
        dcc.Graph(id='feature_importance'),
        dcc.Graph(id='risk_by_type')
    ])
])

# --------------------------------------------------------------------
# Callbacks (Dynamic Interactivity)
# --------------------------------------------------------------------
@app.callback(
    [Output('summary_cards', 'children'),
     Output('risk_dist', 'figure'),
     Output('top_risk_table', 'figure'),
     Output('feature_importance', 'figure'),
     Output('risk_by_type', 'figure')],
    [Input('type_filter', 'value')]
)
def update_dashboard(selected_type):
    df_filtered = df_dashboard[df_dashboard['type'] == selected_type]

    # ---------- KPIs ----------
    total_txn = len(df_filtered)
    high_risk = len(df_filtered[df_filtered['risk_level'] == 'High'])
    avg_risk = df_filtered['risk_score'].mean()

    summary_cards = [
        html.Div([
            html.H4("Total Transactions"),
            html.H2(f"{total_txn:,}")
        ], style={'backgroundColor': colors['card'], 'padding': '20px', 'borderRadius': '12px', 'width': '25%',
                  'textAlign': 'center', 'boxShadow': '0px 0px 10px #111'}),

        html.Div([
            html.H4("High-Risk Transactions"),
            html.H2(f"{high_risk:,}")
        ], style={'backgroundColor': colors['card'], 'padding': '20px', 'borderRadius': '12px', 'width': '25%',
                  'textAlign': 'center', 'boxShadow': '0px 0px 10px #111'}),

        html.Div([
            html.H4("Average Risk Score"),
            html.H2(f"{avg_risk:.2f}")
        ], style={'backgroundColor': colors['card'], 'padding': '20px', 'borderRadius': '12px', 'width': '25%',
                  'textAlign': 'center', 'boxShadow': '0px 0px 10px #111'}),
    ]

    # ---------- Charts ----------

    # 1️⃣ Risk Distribution
    fig1 = px.histogram(
        df_filtered, x='risk_score', nbins=50,
        title='Risk Score Distribution',
        color_discrete_sequence=[colors['accent1']]
    )
    fig1.update_layout(paper_bgcolor=colors['card'], plot_bgcolor=colors['card'], font_color=colors['text'])

    # 2️⃣ Top Risky Transactions
    top10 = df_filtered.sort_values('risk_score', ascending=False).head(10)
    fig2 = px.bar(
        top10, x='nameOrig', y='risk_score', color='risk_score',
        title='Top 10 Risky Transactions', color_continuous_scale='Reds'
    )
    fig2.update_layout(paper_bgcolor=colors['card'], plot_bgcolor=colors['card'], font_color=colors['text'])

    # 3️⃣ Feature Importance
    feat_importance = pd.Series(rf.feature_importances_, index=features).sort_values(ascending=True)
    fig3 = px.bar(
        x=feat_importance.values, y=feat_importance.index, orientation='h',
        title='Feature Importance', color=feat_importance.values,
        color_continuous_scale='Blues'
    )
    fig3.update_layout(paper_bgcolor=colors['card'], plot_bgcolor=colors['card'], font_color=colors['text'])

    # 4️⃣ Risk by Transaction Type
    fig4 = px.box(df_dashboard, x='type', y='risk_score', color='type',
                  title='Risk Score by Transaction Type', color_discrete_sequence=px.colors.qualitative.Vivid)
    fig4.update_layout(paper_bgcolor=colors['card'], plot_bgcolor=colors['card'], font_color=colors['text'])

    return summary_cards, fig1, fig2, fig3, fig4




In [ ]:

app.run_server(mode='external', port=8051)
